In [206]:
import os
os.path.abspath("")

'/home/yuxiangliao/PhD'

# Install Metamap

Follow the following instructions:
- Install Metamap2020: https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/documentation/Installation.html 
- Install additional datasets (2022 Specialist Lexicon, 2022AA UMLS NLM Datasets): https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/additional-tools/DataSetDownload.html

# Processing

## Check if the servers started
- taggerServer
- DisambiguatorServer

In [207]:
import os
cmd = 'ps -ef | grep java'
out = os.popen(cmd)
print(out.read())

yuxiang+    6074    1341  0 Jun25 ?        00:03:55 java -Dtaggerserver.port=1795 -DlexFile=/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/data/lexDB.serial -DngramOne=/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/data/ngramOne.serial -cp /home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/Tagger_server/lib/taggerServer.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/Tagger_server/lib/mps.jar taggerServer
yuxiang+    6108    1341  0 Jun25 ?        00:04:40 java -Xmx2g -Dserver.config.file=/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/config/disambServer.cfg -classpath /home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/metamapwsd.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/utils.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/lucene-core-3.0.1.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/monq-1.1.1.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/wsd.jar:/home/

## Load data

## Run command

In [194]:
# Documentation: https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/MM_2016_Usage.pdf
def get_metamap_command():
    command = format_command_arg("metamap")
    command += format_command_arg("-V NLM")                # Data Version: -V (--mm data version) [Base, USAbase, NLM]
    command += format_command_arg("-Z 2022AA")             # Knowledge Source: -Z (--mm data year)
    command += format_command_arg("--silent")              # Hide Header Output: --silent
    command += format_command_arg("-C")                    # Data Model: [-A (--strict model), -C (--relaxed model)]
    command += format_command_arg("--JSONn")               # Output format: [-q (--machine output), --XMLf, --XMLn, --XMLf1, --XMLn1, --JSONf, --JSONn, -N (--fielded mmi output), -F (--formal tagger output)]
    # command += " --conj"                                   # Turn on Conjunction Processing
    # command += " -y"                                       # Word-Sense Disambiguation: -y (--word sense disambiguation)
    UDA_path = "/home/yuxiangliao/PhD/UMLS/custom-resources/custom-word-replacement"
    command += format_command_arg(f"--UDA {UDA_path}")     # User-Defined Acronyms/Abbreviations (word replacement): --UDA <file>
    # semantic_types = "virs,cgab,acab,ffas,bpoc,medd,tmco,qlco,qnco,bsoj,blor,fndg,sosy,topp,ortf,patf,dsyn,inpo"
    # commend += f"-J {semantic_types}"                      # Retain only Concepts with Specified Semantic Types: -J (--restrict to sts) <list>
    # command += format_command_arg("-I")                    # For human readable output
    return command

def format_command_arg(arg):
    return " " + arg

In [201]:
import os
import subprocess, shlex, time
record = repr("""s222222|PA and lateral views of the chest provided.   The lungs are adequately aerated. 
 There is a focal consolidation at the left lung base adjacent to the lateral hemidiaphragm. There is mild vascular engorgement. There is bilateral apical pleural thickening. 
 The cardiomediastinal silhouette is remarkable for aortic arch calcifications. The heart is top normal in size.
""")
input_command = f"echo -e {record}"
input_process = subprocess.Popen(shlex.split(input_command), stdout=subprocess.PIPE)
meta_command = get_metamap_command()
metamap_process = subprocess.Popen(shlex.split(meta_command), stdout=subprocess.PIPE, stdin=input_process.stdout)

output, error = metamap_process.communicate()

with open("/home/yuxiangliao/PhD/output/metamap.out",'w') as f:
    # Only the second line is the required JSON string.
    f.write(list(output.decode().split("\n"))[1])

### WARNING: Overriding default model 2020AA with 2022AA.
Reading default AAs from file /home/yuxiangliao/PhD/UMLS/custom-resources/custom-word-replacement
